In [1]:
# ref: https://www.kaggle.com/jacklinggu/lstm-with-glove-embedding-public-lb-score-0-049

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from models_def import cnn2d,cnn_v1,cnn_v2,cudnn_gru,lstm_v1,cnn_gru,gru_v1
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing import text, sequence

model_defines = [
#     [cnn2d,'cnn2d'],
#     [cnn_v1,'cnn_v1'],
#     [cnn_v2,'cnn_v2'],
    [cudnn_gru,'cudnn_gru'],
    [lstm_v1,'lstm_v1'],
    [cnn_gru,'cnn_gru'],
    [gru_v1,'gru_v1']
]

max_features = 180000
maxlen = 150

def clean_text( text ):
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    #
    return text

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

list_sentences_train = train["comment_text"].fillna("CVxTz").apply(clean_text).values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").apply(clean_text).values
print(y.shape)

Using TensorFlow backend.


(159571, 6)


In [ ]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

print(X_train.shape,X_test.shape)

# check word_index
tmp_cnt = 0
for k in tokenizer.word_index:
    print(k,tokenizer.word_index[k])
    tmp_cnt += 1
    if tmp_cnt >5:
        break
word_idx = tokenizer.word_index

# read word2vec
# 
word_vec_dict = {}
with open('../crawl-300d-2M.vec') as f:
    first_line_flag = True
    for line in f:
        if first_line_flag:
            first_line_flag= False
            continue
        v_list = line.rstrip().split(' ')
        k = str(v_list[0])
        v = np.array([float(x) for x in v_list[1:]])
        word_vec_dict[k] = v
print(len(word_vec_dict))
print('Preparing embedding matrix')

EMBEDDING_DIM = 300
nb_words = min(max_features,len(word_idx))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word,i in word_idx.items():
    if i >= max_features:
        continue
    else:
        if word in word_vec_dict:
            embedding_matrix[i] = word_vec_dict[word]
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
del word_vec_dict

(159571, 150) (153164, 150)
diskusionoj 103383
bircham 75166
ter 20310
aren 13185
baldwin 13251
ketchup 23801
2000000
Preparing embedding matrix
Null word embeddings: 79399


In [ ]:
from sklearn.metrics import log_loss,accuracy_score
from sklearn.model_selection import KFold
from keras import backend as K
import pickle
from keras.models import load_model
import gc

def eval_val(y,train_x):
    res = 0
    acc_res = 0
    for i in range(6):
        curr_loss = log_loss(y[:,i],train_x[:,i])
        acc = accuracy_score(y[:,i],train_x[:,i].round())
        print(i,curr_loss,acc)
        res += curr_loss
        acc_res += acc
    print('final',res/6, acc_res/6)

def kf_train(model_func,fold_cnt=3,rnd=1,epo=10,batch=64):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((159571,6)),np.zeros((153164,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        
        # model
        model = model_func(maxlen,nb_words,EMBEDDING_DIM,embedding_matrix,trainable_flag=False,comp=True)
        batch_size = batch
        epochs = epo
        file_path="best_model.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit(curr_x, curr_y, batch_size=batch_size, epochs=epochs, 
                  validation_data=(hold_out_x,hold_out_y), callbacks=callbacks_list)
        model = load_model(file_path)
        y_test = model.predict(X_test)
        test_pred += y_test
        hold_out_pred = model.predict(hold_out_x)
        train_pred[test_index] = hold_out_pred
        
        # clear
        del model
        gc.collect()
        K.clear_session()
        
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    eval_val(y,train_pred)
    return train_pred, test_pred

rnd_init = 1
for model_func,model_name in model_defines:
    for fold_k in [5]:
        print('==================================')
        epo = 10
        batch = 64
        train_pred,test_pred = kf_train(model_func,fold_k,rnd_init,epo,batch)
        output_feat = '../features/fasttext_{}_{}_feat.pkl'.format(model_name,fold_k)
        with open(output_feat,'wb') as fout:
            pickle.dump([train_pred,test_pred],fout)
        print(output_feat,'done')
        rnd_init += 1
        




Train on 127656 samples, validate on 31915 samples
Epoch 1/10
127656/127656 [==============================] - 75s 585us/step - loss: 0.0544 - acc: 0.9799 - val_loss: 0.0445 - val_acc: 0.9823
Epoch 2/10
127656/127656 [==============================] - 73s 573us/step - loss: 0.0464 - acc: 0.9822 - val_loss: 0.0433 - val_acc: 0.9834
Epoch 3/10
127656/127656 [==============================] - 73s 575us/step - loss: 0.0441 - acc: 0.9829 - val_loss: 0.0430 - val_acc: 0.9826
Epoch 4/10
127656/127656 [==============================] - 73s 570us/step - loss: 0.0422 - acc: 0.9833 - val_loss: 0.0435 - val_acc: 0.9831
Epoch 5/10
127656/127656 [==============================] - 73s 570us/step - loss: 0.0405 - acc: 0.9838 - val_loss: 0.0438 - val_acc: 0.9825
Epoch 6/10
127656/127656 [==============================] - 73s 570us/step - loss: 0.0389 - acc: 0.9843 - val_loss: 0.0437 - val_acc: 0.9830
Epoch 7/10
127656/127656 [==============================] - 73s 570us/step - loss: 0.0380 - acc: 0.9846

127657/127657 [==============================] - 73s 574us/step - loss: 0.0363 - acc: 0.9852 - val_loss: 0.0456 - val_acc: 0.9834
Epoch 10/10
127657/127657 [==============================] - 73s 574us/step - loss: 0.0352 - acc: 0.9856 - val_loss: 0.0465 - val_acc: 0.9836
Train on 127657 samples, validate on 31914 samples
Epoch 1/10
127657/127657 [==============================] - 74s 583us/step - loss: 0.0549 - acc: 0.9798 - val_loss: 0.0431 - val_acc: 0.9830
Epoch 2/10
127657/127657 [==============================] - 74s 583us/step - loss: 0.0468 - acc: 0.9819 - val_loss: 0.0424 - val_acc: 0.9832
Epoch 3/10
127657/127657 [==============================] - 73s 575us/step - loss: 0.0438 - acc: 0.9830 - val_loss: 0.0431 - val_acc: 0.9826
Epoch 4/10
127657/127657 [==============================] - 73s 575us/step - loss: 0.0419 - acc: 0.9834 - val_loss: 0.0427 - val_acc: 0.9836
Epoch 5/10
127657/127657 [==============================] - 73s 574us/step - loss: 0.0405 - acc: 0.9838 - val_los

Epoch 7/10
127656/127656 [==============================] - 39s 303us/step - loss: 0.0399 - acc: 0.9838 - val_loss: 0.0428 - val_acc: 0.9833
Epoch 8/10
127656/127656 [==============================] - 39s 308us/step - loss: 0.0394 - acc: 0.9839 - val_loss: 0.0434 - val_acc: 0.9830
Epoch 9/10
127656/127656 [==============================] - 39s 303us/step - loss: 0.0391 - acc: 0.9843 - val_loss: 0.0435 - val_acc: 0.9827
Epoch 10/10
127656/127656 [==============================] - 35s 274us/step - loss: 0.0385 - acc: 0.9843 - val_loss: 0.0441 - val_acc: 0.9829
Train on 127657 samples, validate on 31914 samples
Epoch 1/10
127657/127657 [==============================] - 33s 257us/step - loss: 0.0548 - acc: 0.9800 - val_loss: 0.0448 - val_acc: 0.9825
Epoch 2/10
127657/127657 [==============================] - 33s 260us/step - loss: 0.0469 - acc: 0.9820 - val_loss: 0.0427 - val_acc: 0.9826
Epoch 3/10
127657/127657 [==============================] - 33s 259us/step - loss: 0.0449 - acc: 0.982

Epoch 6/10
127657/127657 [==============================] - 34s 266us/step - loss: 0.0409 - acc: 0.9835 - val_loss: 0.0436 - val_acc: 0.9826
Epoch 7/10
127657/127657 [==============================] - 34s 268us/step - loss: 0.0402 - acc: 0.9837 - val_loss: 0.0436 - val_acc: 0.9831
Epoch 8/10
127657/127657 [==============================] - 34s 267us/step - loss: 0.0397 - acc: 0.9840 - val_loss: 0.0430 - val_acc: 0.9833
Epoch 9/10
127657/127657 [==============================] - 34s 267us/step - loss: 0.0390 - acc: 0.9843 - val_loss: 0.0437 - val_acc: 0.9831
Epoch 10/10
127657/127657 [==============================] - 34s 264us/step - loss: 0.0385 - acc: 0.9844 - val_loss: 0.0442 - val_acc: 0.9828
Train on 127657 samples, validate on 31914 samples
Epoch 1/10
127657/127657 [==============================] - 34s 270us/step - loss: 0.0550 - acc: 0.9799 - val_loss: 0.0432 - val_acc: 0.9833
Epoch 2/10
127657/127657 [==============================] - 34s 266us/step - loss: 0.0468 - acc: 0.982

127657/127657 [==============================] - 48s 373us/step - loss: 0.0368 - acc: 0.9850 - val_loss: 0.0435 - val_acc: 0.9834
Epoch 5/10
127657/127657 [==============================] - 47s 370us/step - loss: 0.0343 - acc: 0.9859 - val_loss: 0.0446 - val_acc: 0.9832
Epoch 6/10
127657/127657 [==============================] - 47s 368us/step - loss: 0.0325 - acc: 0.9867 - val_loss: 0.0469 - val_acc: 0.9812
Epoch 7/10
127657/127657 [==============================] - 47s 370us/step - loss: 0.0303 - acc: 0.9875 - val_loss: 0.0479 - val_acc: 0.9832
Epoch 8/10
127657/127657 [==============================] - 47s 369us/step - loss: 0.0288 - acc: 0.9882 - val_loss: 0.0490 - val_acc: 0.9823
Epoch 9/10
127657/127657 [==============================] - 47s 369us/step - loss: 0.0273 - acc: 0.9887 - val_loss: 0.0499 - val_acc: 0.9826
Epoch 10/10
127657/127657 [==============================] - 47s 368us/step - loss: 0.0263 - acc: 0.9892 - val_loss: 0.0529 - val_acc: 0.9824
Train on 127657 samples

127657/127657 [==============================] - 47s 368us/step - loss: 0.0398 - acc: 0.9839 - val_loss: 0.0452 - val_acc: 0.9833
Epoch 4/10
127657/127657 [==============================] - 47s 371us/step - loss: 0.0365 - acc: 0.9850 - val_loss: 0.0446 - val_acc: 0.9829
Epoch 5/10
127657/127657 [==============================] - 47s 368us/step - loss: 0.0342 - acc: 0.9858 - val_loss: 0.0459 - val_acc: 0.9826
Epoch 6/10
127657/127657 [==============================] - 47s 367us/step - loss: 0.0320 - acc: 0.9869 - val_loss: 0.0474 - val_acc: 0.9828
Epoch 7/10
127657/127657 [==============================] - 47s 369us/step - loss: 0.0304 - acc: 0.9873 - val_loss: 0.0492 - val_acc: 0.9828
Epoch 8/10
127657/127657 [==============================] - 47s 368us/step - loss: 0.0288 - acc: 0.9882 - val_loss: 0.0505 - val_acc: 0.9824
Epoch 9/10
127657/127657 [==============================] - 47s 371us/step - loss: 0.0275 - acc: 0.9886 - val_loss: 0.0538 - val_acc: 0.9828
Epoch 10/10
127657/12765